# Approximate states
Compared to exact-state theory, the first and foremost difference is that, in approximate-state methods, state vectors are in general not eigenstates of the unperturbed Hamiltonian but are rather formed in terms of Slater determinants. Apart from this, the steps to be taken in the derivation of response functions are very similar in the two situations.

## SCF approximation
In Hartree--Fock and Kohn--Sham density functional theory, the reference state is described by a single Slater determinant, also referred to as the self-consistent field (SCF) approximation. The detailed derivation of response functions in this approximation can be found in {cite}`Norman2018` but the key steps are indicated below.

### Parameterization
Parameterization of SCF states:

\begin{equation*}
\label{psi-kappa-param}
  | \bar{\psi}(t) \rangle =
    e^{-i\hat{\kappa}(t)} |0\rangle ; \quad
  \hat{\kappa}(t) =
  \sum_s^\mathrm{unocc} \sum_i^\mathrm{occ} \left[
    \kappa_{si}(t) \hat{a}^\dagger_s \hat{a}_i + \kappa_{si}^*(t) \hat{a}^\dagger_i \hat{a}_s
    \right] ,
\end{equation*}

where the creation, $\hat{a}^\dagger$, and annihilation, $\hat{a}$, operators act on unoccupied, secondary, $s$ and occupied, inactive, $i$ molecular orbitals.

### Equation of motion
The time evolution of the SCF state is determined from the Ehrenfest theorem:

\begin{equation*}
\label{ehren-equiv}
  \frac{\partial}{\partial t} 
  \langle \bar{\psi}(t) | \hat{\Omega}_{pq}^\dagger | \bar{\psi}(t) \rangle 
  -
  \langle \bar{\psi}(t) | 
  \frac{\partial \hat{\Omega}_{pq}^\dagger}{\partial t} 
  | \bar{\psi}(t) \rangle = 
  \frac{1}{i\hbar}
    \langle \bar{\psi}(t) | [\hat{\Omega}_{pq}^\dagger, \hat{H}] 
    | \bar{\psi}(t) \rangle 
\end{equation*}

We require that this equation is fulfilled for the set of time-transformed electron-transfer operators

\begin{equation*}
    \hat{\Omega}_{si}^\dagger = 
  e^{-i\hat{\kappa}} \, \hat{a}^\dagger_s \hat{a}_i \, e^{i\hat{\kappa}}
\end{equation*}

as well as the corresponding Hermitian conjugate. There will be two equations associated with each single-electron excited determinant, so there are twice as  many equations as there are unknown *complex* parameters, $\kappa_{si}(t)$, with independent real and imaginary parts in the time-dependent phase-isolated wave function, $| \bar{\psi}(t) \rangle$.

At this point, we adopt perturbation theory:

\begin{equation*}
  \kappa_{si}(t) = \kappa_{si}^{(1)} + \kappa_{si}^{(2)} + \kappa_{si}^{(3)} + \cdots ,
\end{equation*}

which, after finding a first-order solution in the frequency domain, will eventually results in

\begin{equation*}
  \kappa^{(1)}(\omega) = - i
  \left(E^{[2]} - \hbar \omega S^{[2]}\right)^{-1} 
  V^{\omega, [1]} .
\end{equation*}

The vector $\kappa^{(1)}$ collects the set of parameters and their complex conjugate and matrices $E^{[2]}$ and $S^{[2]}$ are known as the electronic Hessian and overlap matrices, respectively, and $V^{\omega, [1]}$ is known as the property gradient of the perturbation operator.

### Response functions
 The response functions of an observable $\hat{\Omega}$ are defined by:

\begin{equation*}
  \langle \bar{\psi}(t) | \hat{\Omega} | \bar{\psi}(t) \rangle =
  \langle 0 | \hat{\Omega} | 0 \rangle + 
  \sum_{\omega}
  \langle \langle \hat{\Omega}; \hat{V}^{\omega} \rangle \rangle 
  F^{\omega}
  e^{-i\omega t} + \cdots 
\end{equation*}

and we are able to identify the following expression for the linear response function

\begin{equation*}
  \langle \langle \hat{\Omega}; \hat{V}^{\omega} \rangle
  \rangle  = -
\left[\Omega^{[1]}\right]^\dagger
   \left(E^{[2]} - \hbar \omega S^{[2]}\right)^{-1} V^{\omega, [1]} ,
\end{equation*}

where $\Omega^{[1]}$ (analogously to  $V^{\omega, [1]}$) is known as the property gradients associated with the observable. This expression for the linear response functions is also known as the random phase approximation (RPA) equation in the case a Hartree--Fock reference state. It is the principal equation to be solved in time-dependent Hartree--Fock (TDHF) and time-dependent density functional theory (TDDFT) theories. Explicit expressions for the vector matrix elements in the TDHF approximaiton are given below.

### Matrix elements
#### Electronic Hessian
The electronic Hessian reads

\begin{eqnarray*}
E^{[2]} & = &
	\begin{pmatrix}
    \rule[-5pt]{0pt}{20pt}
	\rule{5pt}{0pt}
	A & B \rule{5pt}{0pt}
	\\
    \rule[-10pt]{0pt}{25pt}
    \rule{5pt}{0pt}    
	B^* & A^* \\
	\end{pmatrix} = -
  \begin{pmatrix}
    \rule[-10pt]{0pt}{20pt}
     \langle 0 | [\hat{q}_{n},
       [\hat{q}_{m}^\dagger, \hat{H}_0]] | 0 \rangle
     &
     \langle 0 | [\hat{q}_{n},
       [\hat{q}_{m}, \hat{H}_0]] | 0 \rangle \\
    \rule[-5pt]{0pt}{20pt}
     \langle 0 | [\hat{q}_{n}^\dagger,
       [\hat{q}_{m}^\dagger, \hat{H}_0]] | 0 \rangle
     &
     \langle 0 | [\hat{q}_{n}^\dagger,
       [\hat{q}_{m}, \hat{H}_0]] | 0 \rangle \\
  \end{pmatrix}
  \\
&	= &
    \begin{pmatrix}
    \rule[-10pt]{0pt}{30pt}
    \langle 0_i^s | \hat{H}_0 | 0_j^t \rangle - 
    \langle 0 | \hat{H}_0 | 0 \rangle \delta_{st}\delta_{ij} 
    & 
     -\langle 0_{ij}^{st} | \hat{H}_0 | 0 \rangle 
    (1 - \delta_{ij})(1-\delta_{st}) 
    \\
    \rule[-10pt]{0pt}{30pt}
    -\langle 0 | \hat{H}_0 | 0_{ij}^{st} \rangle 
    (1 - \delta_{ij})(1-\delta_{st}) 
     & 
    \langle 0_j^t | \hat{H}_0 | 0_i^s \rangle - 
    \langle 0 | \hat{H}_0 | 0 \rangle \delta_{st}\delta_{ij} 
    \\
  \end{pmatrix} 
\end{eqnarray*}

- The Hessian matrix is Hermitian and has an internal structure in terms of the matrix blocks $A$ and $B$
- The diagonal elements correspond to differences in energy between the single-excited determinants $|0_i^s\rangle$ and the reference state $|0\rangle$
- The $A$-block corresponds to the CIS approximation

In the Hartree–Fock approximation, the Hessian can be expressed as the differences in orbital energies corrected by an anti-symmetrized two-electron integral

$$
E^{[2]} = 
\begin{pmatrix}
    \rule[-10pt]{0pt}{30pt}
    F_{st} \delta_{ij} \!-\! F_{ji}\delta_{st} \!-\! \langle sj || ti
    \rangle 
    & 
    -\langle st||ij \rangle 
    \\
    \rule[-10pt]{0pt}{30pt}
    -\langle ij||st \rangle 
    & 
    F_{ts} \delta_{ij}\! -\! F_{ij} \delta_{st} 
    \! - \! \langle ti || sj \rangle 
    \\
\end{pmatrix}
$$

- The anti-symmetrized ERI gives the correct asymptotic behavior for charge-transfer excitations
- At the level of DFT, the exchange–correlation functional is introduced and the asymptotic behavior becomes incorrect

#### Overlap matrix

The overlap matrix in the SCF approximation is trivial and equal to

\begin{eqnarray*}
\label{S2}
S^{[2]} & = &
  \begin{pmatrix}
    \rule[-10pt]{0pt}{20pt}
     \langle 0 | [\hat{q}_{n}, \hat{q}_{m}^\dagger] | 0 \rangle
     &
     \langle 0 | [\hat{q}_{n}, \hat{q}_{m}] | 0 \rangle \\
    \rule[-5pt]{0pt}{20pt}
     \langle 0 | [\hat{q}_{n}^\dagger, \hat{q}_{m}^\dagger] | 0 \rangle
     &
     \langle 0 | [\hat{q}_{n}^\dagger, \hat{q}_{m}] | 0 \rangle \\
  \end{pmatrix} =
	\begin{pmatrix}
    \rule[-5pt]{0pt}{20pt}
	\rule{5pt}{0pt}
	1 & 0 \rule{5pt}{0pt}
	\\
    \rule[-10pt]{0pt}{25pt}
    \rule{5pt}{0pt}    
	0 & -1 \\
	\end{pmatrix} ,
\end{eqnarray*}

where the "1" and the "0" are here understood to be the identity and zero matrices, respectively, of the same rank as sub-blocks A and B in the electronic Hessian.

#### Property gradient

In the SCF approximation, the property gradient takes the form

\begin{equation*}
  V^{[1]} =
  \begin{pmatrix}
    \rule[-10pt]{0pt}{20pt}
     \langle 0 | [\hat{a}_{i}^\dagger\hat{a}_{s}, \hat{V}] | 0 \rangle
       \\
    \rule[-5pt]{0pt}{20pt}
     \langle 0 | [\hat{a}_{s}^\dagger\hat{a}_{i}, \hat{V}] | 0 \rangle
  \\
  \end{pmatrix} =
  \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
     \langle 0_i^s | \hat{V} | 0 \rangle
       \\
    \rule[-5pt]{0pt}{0pt}
     -\langle 0 | \hat{V} | 0_i^s \rangle
    \\
  \end{pmatrix} =
      \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
    g
       \\
    \rule[-5pt]{0pt}{0pt}
     -g^*
\\
  \end{pmatrix}
\end{equation*}

```{note}
It is a common practice to adopt the use of spin-adapted electron excitation vectors as a means to reduce the dimension of matrices and vectors. The expressions for the matrix elements then become modified accordingly.
```

#### Numerical example
Let us consider ethylene in a minimal basis set. We perform an SCF optimization of the Hartree--Fock reference state with use of the VeloxChem program. The dictonary `scf_tensors` contains information of the SCF calculation and we retrieve the orbital energies.

In [77]:
from mpi4py import MPI
comm = MPI.COMM_WORLD

import veloxchem as vlx
ostream = vlx.OutputStream('/dev/null') # we will suppress program output

import numpy as np

In [119]:
mol_str = """
C        0.67759997    0.00000000    0.00000000
C       -0.67759997    0.00000000    0.00000000
H        1.21655197    0.92414474    0.00000000
H        1.21655197   -0.92414474    0.00000000
H       -1.21655197   -0.92414474    0.00000000
H       -1.21655197    0.92414474    0.00000000
"""
molecule = vlx.Molecule.read_str(mol_str, units='angstrom')
basis = vlx.MolecularBasis.read(molecule, "sto-3g")

scf_drv = vlx.ScfRestrictedDriver(comm, ostream)
scf_drv.compute(molecule, basis)

orbital_energies = scf_drv.scf_tensors['E']
C = scf_drv.scf_tensors['C'] # MO coefficients

##### E[2] matrix
For a closed-shell system, the dimension of the $A$- and $B$-blocks of the $E^{[2]}$-matrix equals the number of occupied (spatial) orbitals times the number of virtual orbitals or, in other words

$$\mathrm{dim}\left(E^{[2]}\right) = 2 \times n_\mathrm{occ} \times n_\mathrm{virt}$$

In [85]:
nocc = molecule.number_of_alpha_electrons()
norb = orbital_energies.shape[0]
nvirt = norb - nocc
n = nocc * nvirt

print('Number of occupied orbitals:', nocc)
print('Number of virtual orbitals:', nvirt)
print('Dimension of A-block in E2:', n)

Number of occupied orbitals: 8
Number of virtual orbitals: 6
Dimension of A-block in E2: 48


We get the $E^{[2]}$-matrix from the `LinearResponseEigenSolver` class object.

In [142]:
lres_drv = vlx.LinearResponseEigenSolver(comm, ostream)
E2 = lres_drv.get_e2(molecule, basis, scf_drv.scf_tensors)

# index position for (pi,pi*) excitation
idx = (nocc -1) * nvirt

print('Dimension of E[2] matrix:', E2.shape)
print(f'Diagonal element for the (pi,pi*)-excitation: {E2[idx,idx] : .6f} a.u.')

Dimension of E[2] matrix: (96, 96)
Diagonal element for the (pi,pi*)-excitation:  0.935688 a.u.


Alternatively, we can in principle determine the elements of the $E^{[2]}$-matrix from explicit calculations using orbital energies and two-electron integrals in the MO-basis.

In [87]:
# ERI integrals in physicist's notation
moints_drv = vlx.MOIntegralsDriver()
isis = moints_drv.compute_in_mem(molecule, basis, scf_drv.mol_orbs, "OVOV")
iiss = moints_drv.compute_in_mem(molecule, basis, scf_drv.mol_orbs, "OOVV")
# Determine the specific diagonal matrix element of E[2] explicitly with
# account made for use of spin-adapted excitation operators
print(f'Diagonal element (explicit calculation): {2*(orbital_energies[8] - orbital_energies[7]- isis[7,0,7,0] + 2 * iiss[7,7,0,0]) : .6f} a.u.')

Diagonal element (explicit calculation):  0.935688 a.u.


##### S[2] matrix
We construct the trivial $S^{[2]}$-matrix.

In [88]:
# factor of 2 due to spin-adaptation
S2 = 2 * np.identity(2 * n)
S2[n:,n:] *= -1

print('Dimension of S[2] matrix:', S2.shape)

Dimension of S[2] matrix: (96, 96)


##### V[1] gradient
We get the $V^{\omega,[1]}$-vector from the `LinearResponseSolver` class object. We assume the perturbation operator to equal minus the electric dipole moment operator along the $x$-axis, i.e., the molecular C--C axis.

In [144]:
lrs_drv = vlx.LinearResponseSolver(comm, ostream)

mu_x = lrs_drv.get_prop_grad('electric dipole', 'x', molecule, basis, scf_drv.scf_tensors)[0]
mu_y = lrs_drv.get_prop_grad('electric dipole', 'y', molecule, basis, scf_drv.scf_tensors)[0]
mu_z = lrs_drv.get_prop_grad('electric dipole', 'z', molecule, basis, scf_drv.scf_tensors)[0]

V1 = -mu_x

print('Dimension of V[1] vector:', V1.shape[0])
print('Elements for (pi,pi*)-excitation:')
print(f'(upper)   g: {V1[idx] : .6f}')
print(f'(lower) -g*: {V1[n + idx] : .6f}')

Dimension of V[1] vector: 96
Elements for (pi,pi*)-excitation:
(upper)   g: -2.632098
(lower) -g*:  2.632098


Alternatively, we can in principle determine the elements of the $V^{\omega,[1]}$-vector from explicit calculations using one-electron integrals of the perturbation operator in the MO-basis. The property integrals are available in AO-basis from the `ElectricDipoleIntegralsDriver` class object and we transfer them to the MO-basis with use of the MO-coefficients.

In [133]:
dipole_drv = vlx.ElectricDipoleIntegralsDriver()

dipole_matrices = dipole_drv.compute(molecule, basis)
x_ao = dipole_matrices.x_to_numpy()
x_mo = np.matmul(C.T, np.matmul(x_ao, C))

# factor of 2 due to spin-adaptation
print(f'MO integral 2 * < pi | x | pi* >: {2 * x_mo[7,8] : .6f}')

MO integral 2 * < pi | x | pi* >:  2.632098


### Electric-dipole polarizability
The polarizability is determined from the linear response function according to

$$
\alpha_{\alpha\beta}(\omega) = -
\langle \langle \hat{\mu}_\alpha ; \hat{\mu}_\beta \rangle \rangle_\omega
$$

Here, $\hat{\Omega} = \hat{\mu}_\alpha$ and $\hat{V}^\omega = - \hat{\mu}_\beta$ are associated with the observable and perturbation, respectively.

This linear response function is determined from the RPA matrix

$$
\langle\langle \hat{\mu}; \hat{V}^\omega \rangle\rangle =
-{\mu^{[1]}}^\dagger
\Big(
E^{[2]} - \omega S^{[2]}
\Big)^{-1}
V^{\omega, [1]}
$$

In typical applications, the matrix dimension is large and instead of explicitly constructing the Hessian matrix, we solve the linear response equation with iterative techniques

$$
N(\omega) =
\Big(
E^{[2]} - \omega S^{[2]}
\Big)^{-1}
V^{\omega,[1]}
$$

such that

$$
\langle\langle \hat{\mu}; \hat{V}^\omega \rangle\rangle =
-{\mu^{[1]}}^\dagger N(\omega)
$$

A reduced-space iterative algorithm for solving the linear response funciton is available from the `LinearResponseSolver` class object.

In [148]:
lrs_drv = vlx.LinearResponseSolver(comm, ostream)
lrs_out = lrs_drv.compute(molecule, basis, scf_drv.scf_tensors)

The return result `lrs_out` is a dictonary with the values of the response functions provided from the key `response_functions`.

In [149]:
lrs_out.keys()

dict_keys(['response_functions', 'solutions'])

In [150]:
alpha_xx = -lrs_out['response_functions'][('x', 'x', 0)]
alpha_yy = -lrs_out['response_functions'][('y', 'y', 0)] 
alpha_zz = -lrs_out['response_functions'][('z', 'z', 0)]

print(f"alpha_xx(0): {alpha_xx : .6f}")
print(f"alpha_yy(0): {alpha_yy : .6f}")
print(f"alpha_zz(0): {alpha_zz : .6f}")

alpha_xx(0):  19.356860
alpha_yy(0):  10.842377
alpha_zz(0):  2.557959


The solution vectors (or *response vectors*) are available from the key solutions, so that we can also determine the the linear response function values from the multiplication of response vectors with property gradients.

In [151]:
Nx = lrs_out['solutions'][('x', 0)]
Ny = lrs_out['solutions'][('y', 0)]
Nz = lrs_out['solutions'][('z', 0)]

print(f"alpha_xx(0): {np.dot(mu_x, Nx) : .6f}")
print(f"alpha_yy(0): {np.dot(mu_y, Ny) : .6f}")
print(f"alpha_zz(0): {np.dot(mu_z, Nz) : .6f}")

alpha_xx(0):  19.356860
alpha_yy(0):  10.842377
alpha_zz(0):  2.557959


In this example, the `LinearResponseSolver` was using its default settings for operators and frequencies, both of which can be changed. We change the value of the optical frequency from the default zero to 0.0656 a.u. with use of the dictionary `rsp_settings`.

In [152]:
lrs_drv = vlx.LinearResponseSolver(comm, ostream)
rsp_settings = {'frequencies': [0.0656]}
method_settings = {} # Hartree-Fock is default
lrs_drv.update_settings(rsp_settings, method_settings)

lrs_out = lrs_drv.compute(molecule, basis, scf_drv.scf_tensors)

alpha_xx = -lrs_out['response_functions'][('x', 'x', 0.0656)]
alpha_yy = -lrs_out['response_functions'][('y', 'y', 0.0656)] 
alpha_zz = -lrs_out['response_functions'][('z', 'z', 0.0656)]

print(f"alpha_xx(w=0.0656): {alpha_xx : .6f}")
print(f"alpha_yy(w=0.0656): {alpha_yy : .6f}")
print(f"alpha_zz(w=0.0656): {alpha_zz : .6f}")

alpha_xx(w=0.0656):  19.757304
alpha_yy(w=0.0656):  10.900129
alpha_zz(w=0.0656):  2.577366


Alternatively, in our small example the matrices in the RPA equation ($E^{[2]}$ and $S^{[2]}$) have been formed explicitly and we can thus determine values of linear response functions by straightforward matrix inverse and multiplication.

In [156]:
omega = 0.0656
alpha_xx = np.dot(mu_x, np.matmul(np.linalg.inv(E2 - omega * S2), mu_x))
alpha_yy = np.dot(mu_y, np.matmul(np.linalg.inv(E2 - omega * S2), mu_y))
alpha_zz = np.dot(mu_z, np.matmul(np.linalg.inv(E2 - omega * S2), mu_z))

print(f"alpha_xx(w=0.0656): {alpha_xx : .6f}")
print(f"alpha_yy(w=0.0656): {alpha_yy : .6f}")
print(f"alpha_zz(w=0.0656): {alpha_zz : .6f}")

alpha_xx(w=0.0656):  19.757304
alpha_yy(w=0.0656):  10.900129
alpha_zz(w=0.0656):  2.577366


### Transition energies
We denote by $X_e$ the eigenvectors of the generalized eigenvalue equation

\begin{equation}
  E^{[2]} X_e = \lambda_e S^{[2]} X_e, \quad e = -n,\ldots,-1,1,\ldots,n ,
\end{equation}
 
where the matrix dimension is $2n$. We find the set of eigenvalues and eigenvectors by diagonalizing the non-Hermitian matrix $\left(S^{[2]}\right)^{-1} E^{[2]}$ according to

\begin{equation}
\label{gen-eig}
  X^{-1} \left[ \left(S^{[2]}\right)^{-1} E^{[2]} \right] X = 
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{9pt}{0pt}
    \lambda & \rule{3pt}{0pt} 0 \rule{3pt}{0pt} \\
    \rule{3pt}{0pt}
    0 & -\lambda \rule{3pt}{0pt} \\
  \end{bmatrix} ,
\end{equation}

where $\lambda$ is a diagonal matrix of dimension $n$ collecting the eigenvalues with positive index and the columns of $X$ store the eigenvectors ($X$ is assumed to be nonsingular). This pairing of eigenvalues has its correspondence in the eigenvectors through

\begin{equation*}
  X_e =
  \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
    Z_e \\
    \rule[-5pt]{0pt}{0pt}
    Y_e^* \\
  \end{pmatrix}
\; \mbox{with eigenvalue}\; \lambda_e
; \qquad
  X_{-e} =
  \begin{pmatrix}
    \rule[-5pt]{0pt}{15pt}
    Y_{e} \\
    \rule[-5pt]{0pt}{0pt}
    Z_{e}^* \\
  \end{pmatrix} 
\; \mbox{with eigenvalue}\; -\!\lambda_e .
\end{equation*}

The matrix $X$ will therefore have the structure

\begin{equation}
\label{X-struct}
  X = 
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{3pt}{0pt}
    Z & \rule{3pt}{0pt} Y \rule{3pt}{0pt} \\
    \rule{8pt}{0pt}
    Y^* & \rule{3pt}{0pt} Z^* \rule{3pt}{0pt} \\
  \end{bmatrix} .
\end{equation}

With an appropriate scaling of the eigenvectors $X_e$, the non-unitary matrix $X$ achieves a simultaneous diagonalization of the two non-commuting matrices $E^{[2]}$ and $S^{[2]}$ as

\begin{equation}
\label{sim-diag}
  X^\dagger E^{[2]} X = 
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{9pt}{0pt}
    \lambda & \rule{3pt}{0pt} 0 \rule{3pt}{0pt} \\
    \rule{3pt}{0pt}
    0 & \lambda \rule{3pt}{0pt} \\
  \end{bmatrix} ; \qquad
  X^\dagger S^{[2]} X =
  \begin{bmatrix}
    \rule[-5pt]{0pt}{15pt}
    \rule{8pt}{0pt}
    I & \rule{5pt}{0pt} 0 \rule{3pt}{0pt} \\
    \rule{5pt}{0pt}
    0 & -I \rule{3pt}{0pt} \\
  \end{bmatrix} .
\end{equation}

For each pair of eigenvectors, the second equation involving the metric of the generalized eigenvalue equation defines which of the two that should be indexed with a positive index. We also emphasize that $X^\dagger \neq X^{-1}$ and for this reason the $X_e$'s are eigenvectors neither of $E^{[2]}$ nor of $S^{[2]}$, just as the $\lambda_e$'s are not the eigenvalues of $E^{[2]}$.

In [9]:
eigs, X = np.linalg.eig(np.matmul(np.linalg.inv(S2), E2))
print(eigs)

# the second smallest positive eigenvalue is the sigma-pi* transition
eigs[np.argmin(np.where(eigs>10, eigs, 999))] = 999
pos = np.argmin(np.where(eigs>10, eigs, 999))

Xf = X[:,pos]
Xf =  Xf / np.sqrt(np.matmul(Xf.T,np.matmul(S2,Xf)))

print('Eigenvector position:', pos)
print(f'sigma_u - pi* transition energy: {eigs[pos] : .4f} a.u. (or {eigs[pos] * au_to_ev : .1f} eV)')
print(f'Xf norm under metric S[2] = {np.matmul(Xf.T,np.matmul(S2,Xf)) : .4f}')

[ 10.7286  10.7291 -10.7286 -10.7291  11.5355  11.5355  11.0765 -11.5355 -11.5355 -11.478   11.2445  11.0949  11.0784  11.478
 -11.2445 -11.0784 -11.2449 -11.478  -11.0765 -11.0949  11.0965  11.2449  11.478  -11.0965  -1.5626   1.5626  -1.5137   1.5137
   0.3958  -0.3958  -1.4065  -1.3792   1.4065   1.3792   1.3791  -1.3791   0.3685  -0.3685  -0.6943   1.2962   0.4191   0.6943
   1.3142  -1.3142  -1.2962  -0.4191  -0.5365   0.5365   1.2609  -1.2609   0.598   -0.598   -0.7917  -0.6656   0.6656   0.7917
   1.1817  -0.9081   0.9081  -0.7555  -1.1817   1.1624  -1.166   -1.1624   1.166    0.7555  -1.1062  -1.1474  -1.1382   1.1062
   1.1474   1.1382  -1.0773   1.0773  -1.0282   1.0282  -1.0198   1.0198   1.0059   1.0023  -1.0059   0.8365   0.8396  -0.8245
  -0.8365  -1.0023   0.8245  -0.8396  -0.8772  -0.8717   0.8717   0.8871   0.8772   0.8731  -0.8871  -0.8731]
Eigenvector position: 1
sigma_u - pi* transition energy:  10.7291 a.u. (or  292.0 eV)
Xf norm under metric S[2] =  1.0000


### Transition moments
Since $(\Omega \Lambda)^{-1} = \Lambda^{-1} \Omega^{-1}$, we can form the inverse of the RPA matrix as

\begin{equation}
  \left( E^{[2]} - \omega S^{[2]} \right)^{-1} =
  X \left[ X^\dagger \left( E^{[2]} - \omega S^{[2]} \right) X \right]^{-1} X^\dagger
\end{equation}

which yields an expression for the linear response function that reads

\begin{eqnarray}
\label{lrf-resolved}
  \langle
  \langle
  \hat{\Omega}; \hat{V}
  \rangle
  \rangle & = &
  -
  \left[\Omega^{[1]}\right]^\dagger 
  \sum_{e=1}^n
  \left(
  \frac{
    X_e
    X^\dagger_e
    }{
    \lambda_e - \hbar \omega
    }
  +
  \frac{
    X_e
    X^\dagger_e 
    }{
    \lambda_e + \hbar \omega
    }
  \right) V^{[1]} 
\end{eqnarray}

This equation suggests that we identify the $\lambda$-eigenvalues as excitation energies of the system and transition moments in the SCF approximation are identified as

\begin{equation}
\label{tranmom}
    \langle f | \hat{V} | 0 \rangle =
    X^\dagger_f V^{[1]}
\end{equation}

In [10]:
tmom = np.dot(Xf, V1)
print(f'Transition moment (a.u.): {tmom : .6f}')

Transition moment (a.u.):  0.141356


### Core–valence separation (CVS) approximation

Due to the large energy separation of core and valence electronic states there will be negigiible contribution from the valence-excitaiton channels. 

In [11]:
n = nocc * nvirt
c = ncore * nvirt

E2_CVS = np.zeros((2 * c, 2 * c))

E2_CVS[0:c, 0:c] = E2[0:c, 0:c]
E2_CVS[0:c, c:2*c] = E2[0:c, n:n+c]
E2_CVS[c:2*c, 0:c] = E2[n:n+c, 0:c]
E2_CVS[c:2*c, c:2*c] = E2[n:n+c, n:n+c]

S2_CVS = 2 * np.identity(2*c)
S2_CVS[c:2*c,c:2*c] *= -1

V1_CVS = np.zeros(2*c)
V1_CVS[0:c] = V1[0:c]
V1_CVS[c:2*c] = V1[n:n+c]

eigs_CVS, X_CVS = np.linalg.eig(np.matmul(np.linalg.inv(S2_CVS), E2_CVS))
print(eigs_CVS)

# the second smallest positive eigenvalue is the sigma-pi* transition
eigs_CVS[np.argmin(np.where(eigs_CVS>10, eigs_CVS, 999))] = 999
pos = np.argmin(np.where(eigs_CVS>10, eigs_CVS, 999))

print('Eigenvector position:', pos)
print(f'sigma_u - pi* transition energy in CVS: {eigs_CVS[pos] : .4f} a.u. (or {eigs_CVS[pos] * au_to_ev : .1f} eV)')
Xf_CVS = X_CVS[:,pos]
Xf_CVS =  Xf_CVS / np.sqrt(np.matmul(Xf_CVS.T, np.matmul(S2_CVS, Xf_CVS)))

print(f'Xf norm under metric S[2] = {np.matmul(Xf_CVS.T, np.matmul(S2_CVS, Xf_CVS)) : .4f}')

tmom_CVS = np.dot(Xf_CVS, V1_CVS)
print(f'Transition moment in CVS (a.u.): {tmom_CVS : .6f}')

[ 10.7286  10.7291  11.0764  11.0783  11.0948  11.0964  11.2444  11.2448  11.5355  11.5354  11.478   11.478  -10.7286 -10.7291
 -11.0764 -11.0783 -11.0948 -11.0964 -11.2444 -11.2448 -11.5355 -11.5354 -11.478  -11.478 ]
Eigenvector position: 1
sigma_u - pi* transition energy in CVS:  10.7291 a.u. (or  292.0 eV)
Xf norm under metric S[2] =  1.0000
Transition moment in CVS (a.u.): -0.139695


### Tamm-Dancoff approximation (TDA)

The TDA corresponds to setting $B=0$ in the electronic Hessian and diagonalize matrix block $A$:

\begin{eqnarray}
E^{[2]} & = &
	\begin{pmatrix}
    \rule[-5pt]{0pt}{20pt}
	\rule{5pt}{0pt}
	A & B \rule{5pt}{0pt}
	\\
    \rule[-10pt]{0pt}{25pt}
    \rule{5pt}{0pt}    
	B^* & A^* \\
	\end{pmatrix}
\end{eqnarray}

In [12]:
# divide by factors of 2 and sqrt(2) to account for normalization of spin-adapted excitation operators
E2_CVS_TDA = E2_CVS[0:c, 0:c] / 2
V1_CVS_TDA = V1_CVS[0:c] / np.sqrt(2)

eigs_CVS_TDA, X_CVS_TDA = np.linalg.eigh(E2_CVS_TDA)
print(eigs_CVS_TDA)

pos=1
print(f'sigma_u - pi* transition energy with CVS and TDA (eV): {eigs_CVS_TDA[pos] * au_to_ev : .4f}')
Xf_CVS_TDA = X_CVS_TDA[:,pos]

tmom_CVS_TDA = np.dot(Xf_CVS_TDA, V1_CVS_TDA)
print(f'Transition moment with CVS and TDA (a.u.): {tmom_CVS_TDA : .6f}')

[10.7286 10.7291 11.0764 11.0783 11.0948 11.0965 11.2445 11.2449 11.478  11.478  11.5354 11.5355]
sigma_u - pi* transition energy with CVS and TDA (eV):  291.9529
Transition moment with CVS and TDA (a.u.):  0.139783


### Natural transition orbitals (NTOs)

Collecting occupied and unoccupied (real) MOs as vectors

$$
\bar{\phi}_\mathrm{occ} = (\ldots, \phi_i(\mathbf{r}),  \ldots)^T; \quad
\bar{\phi}_\mathrm{unocc} = (\ldots, \phi_s(\mathbf{r}),  \ldots)^T
$$

and scattering the (real) RPA eigenvector into a rectangular transition matrix $T$ of dimension $n_\mathrm{occ} \times n_\mathrm{unocc}$

$$
T_{f, is} = Z_{f, is} - Y_{f, is}
$$

we can write the transition moment as

\begin{equation}
    \langle f | \hat{V} | 0 \rangle =
    X^T_f V^{[1]} = 
    \big(
    Z_f^T, Y_f^T
    \big)
    \begin{pmatrix}
    g\\-g
    \end{pmatrix}
    =
    \big(
    Z_f - Y_f
    \big)^T
    g
    =
    \int
    \hat{\mu}_\alpha
    \bar{\phi}_\mathrm{occ}^T
    T
    \bar{\phi}_\mathrm{unocc} \;
    d^3\mathbf{r}
\end{equation}

By means of a singular value decomposition (SVD), we can factorize the transition matrix

$$
T = U \Lambda V^T
$$

where matrices $U$ and $V$ are unitary and $\Lambda$ is rectangular diagonal.

The SVD transformation matrices define the pairs of hole and electron NTOs

$$
\bar{\phi}_\mathrm{h}^T = \bar{\phi}_\mathrm{occ}^T U; \quad
\bar{\phi}_\mathrm{e} = V^T \bar{\phi}_\mathrm{unocc}
$$

such that

\begin{equation}
    \langle f | \hat{\mu}_\alpha | 0 \rangle =
    \int
    \hat{\mu}_\alpha
    \bar{\phi}_\mathrm{h}^T
    \Lambda
    \bar{\phi}_\mathrm{e} \;
    d^3\mathbf{r}
\end{equation}

In our CVS example, we get 

$$
\bar{\phi}_\mathrm{occ}^T = (\sigma_g, \sigma_u) ; \quad
\Lambda =
\begin{bmatrix}
 \lambda_1 & 0 & 0 & 0 & 0 & 0 \\
 0 & \lambda_2 & 0 & 0 & 0 & 0 \\
\end{bmatrix}; \quad
\bar{\phi}_\mathrm{unocc}^T = (\pi^*, 2b_{2u}, 4a_{g}, 3b_{3u}, 4b_{3u}, 2b_{1g})
$$

and the associated contributing electron orbital becomes

$$
\phi_\mathrm{e} = \sum_s T_{f,is} \phi_s; \quad i=2
$$

In [13]:
Zf = Xf_CVS[:ncore*nvirt]
Yf = Xf_CVS[ncore*nvirt:]
T = np.reshape(Zf - Yf, (ncore,nvirt))
U, L, VT = np.linalg.svd(T)

print('Unitary matrix for occupied MOs:\n', U)
print('Lambda diagonal elements:\n', L)
print('Unitary matrix for unoccupied MOs:\n', VT)

Unitary matrix for occupied MOs:
 [[ 0. -1.]
 [ 1.  0.]]
Lambda diagonal elements:
 [0.7067 0.    ]
Unitary matrix for unoccupied MOs:
 [[-1.     -0.      0.     -0.      0.     -0.    ]
 [-0.      0.0124 -0.9686  0.1776 -0.1157  0.1292]
 [-0.      0.9686  0.0732  0.1699 -0.1107  0.1236]
 [-0.     -0.1776  0.1699  0.9688  0.0203 -0.0227]
 [ 0.      0.1157 -0.1107  0.0203  0.9868  0.0148]
 [-0.     -0.1292  0.1236 -0.0227  0.0148  0.9835]]
